## Problem 85: Counting rectangles

By counting carefully it can be seen that a rectangular grid measuring 3 by 2 contains eighteen rectangles:

![title](p085.png)

Although there exists no rectangular grid that contains exactly two million rectangles, find the area of the grid with the nearest solution.

## Answer

This problem can be solved in two step: first, create a function whose inputs are the length $n$ and width $m$ of rectangles and returns the counting of sub-rectangles it contains; second, run a loop to find those rectangles whose counting nears $2 \times 10^6$, and find the closest one from them.

For the first step, since each rectangle contains $n \times m$ types sub-rectangles, and it is easy to show that the number of each type of sub-rectangles (whose length is $i$, width is $j$) is $(n - (i - 1))\times(m - (j - 1))$ (you can try it on a small rectangles, e.g. the examples in the question). We just need to create a $n \times m$ matrix whose elements are the number of each sub-rectangle, and sum all the elements.

In [1]:
import pandas as pd
import numpy as np

def counting_rect(n, m):
    '''
    Counting the number of sub-rectangles in a given rectangle

    Args:
        n : The length of rectangle
        m : The width of rectangle

    Returns:
        The number of sub-rectangles
    '''
    rect = np.zeros([n,m])
    for i in range(0, n):
        for j in range(0, m):
            rect[i,j] = (n - i) * (m - j)
            
    return np.sum(rect)

Then, we need to find which rectangle's counting is closest to $2 \times 10^6$. After some trials, I found that the counting of $1 \times 2000$ was sliently above $2 \times 10^6$, so the answer must lie in the range of $[1, 2000] \times [1, 2000]$, but it was still a wide range. I narrowed it down by starting trial from $n = 1$, as long as counting hits $2 \times 10^6$, turn to a higher $n$. It is obvious that the `counting_rect` function is symmetric in $n, m$, so when hiting $2 \times 10^6$, as long as $n > m$, the following loop is just repetition of the former. With these two constrains, we can sharply reduce our work and get answer faster. 

In [2]:
near_2e6 = []

L = 2000
k = 1
while k <= L:
    l = 1
    while counting_rect(k, l) < 2e6:
        l = l + 1
    near_2e6.append([k, l - 1, counting_rect(k, l - 1)])
    near_2e6.append([k, l , counting_rect(k, l)])
    L = l
    k = k + 1

The remaining work is just to find the closest one from what we found in above steps.

In [3]:
near_2e6 = pd.DataFrame(near_2e6).astype(int)
near_2e6

,0,1,2
0,1,1999,1999000
1,1,2000,2001000
2,2,1154,1999305
3,2,1155,2002770
4,3,815,1995120
...,...,...,...
101,51,55,2042040
102,52,53,1971918
103,52,54,2046330
104,53,52,1971918


In [4]:
diff = abs(near_2e6[2] - 2e6)
min_idx = diff.argmin(axis = 0)
near_2e6[min_idx:min_idx + 1]

,0,1,2
70,36,77,1999998


In [5]:
area = 36 * 77
area

2772

## Reference
1. https://www.mathblog.dk/project-euler-85-rectangles-rectangular-grid/